### 自动微分

计算机求导方法
对计算机程序求导的方法可以归纳为以下四种：

**手动求解法(Manual Differentiation) **：根据链式求导法则，手工求导并编写对应的结果程序，依据链式法则解出梯度公式，带入数值，得到梯度。

数值微分法(Numerical Differentiation)：利用导数的原始定义，通过有限差分近似方法完成求导，直接求解微分值。

符号微分法(Symbolic Differentiation)：基于数学规则和程序表达式变换完成求导。利用求导规则对表达式进行自动计算，其计算结果是导函数的表达式而非具体的数值。即，先求解析解，然后转换为程序，再通过程序计算出函数的梯度。

自动微分法(Automatic Differentiation)：介于数值微分和符号微分之间的方法，采用类似有向图的计算来求解微分值，也是本文介绍的重点。

而自动微分则是分为前向微分和后向微分两种实现模式，不同的实现模式有不同的机制和计算逻辑

### 雅各比矩阵

在向量微积分中，**Jacobian 矩阵是**一阶偏导数以一定方式排列成的矩阵，其行列式称为 Jacobian 行列式。Jacobian 矩阵的重要性在于它体现了一个可微方程与给出点的最优线性逼近。

Jacobian 矩阵表示两个向量所有可能的偏导数。它是一个向量相对于另一个向量的梯度，其实现的是 $n$ 维向量到 $m$ 维向量的映射。

在矢量运算中，Jacobian 矩阵是基于函数对所有变量一阶偏导数的数值矩阵，当输入个数等于输出个数时又称为 **Jacobian** 行列式。

假设输入向量 $𝑥∈𝑅_n$，而输出向量 $𝑦∈𝑅_m$，则 Jacobian 矩阵定义为：

$$
J_f= \left[ \begin{matrix} \dfrac{\delta y_1}{\delta x_1} & \cdots & \dfrac{\delta y_1}{\delta x_n} \\ \vdots & \ddots & \vdots \\ \dfrac{\delta y_m}{\delta x_1} & \vdots & \dfrac{\delta y_m}{\delta x_n} \end{matrix} \right]
$$

自动微分理论补充

https://github.com/chenzomi12/AISystem/blob/main/05Framework/02AutoDiff/03GradMode.md

## 自动微分实现
预定义了特定的数据结构，并对该数据结构重载了相应的基本运算操作符；
程序在实际执行时会将相应表达式的操作类型和输入输出信息记录至特殊数据结构；
得到特殊数据结构后，将对数据结构进行遍历并对其中记录的基本运算操作进行微分；
把结果通过链式法则进行组合，完成自动微分。

### 1.前向微分

前向自动微分关键步骤为：

分解程序为一系列已知微分规则的基础表达式的组合
根据已知微分规则给出各基础表达式的微分结果
根据基础表达式间的数据依赖关系，使用链式法则将微分结果组合完成程序的微分结果

In [41]:
import numpy as np

class ADTangent:
    # 自变量 x 和 导数 dx
    def __init__(self, x, dx):
        self.x = x
        self.dx = dx
    
    def __str__(self):
        context = f'value: {self.x}, grad: {self.dx}'
        return context

    def __add__(self, other):
        # 判断操作数是否为 ADTangent 类型
        if isinstance(other, ADTangent):
            x = self.x + other.x
            dx = self.dx + other.dx
        elif isinstance(other, (int, float)):
            x = self.x + other
            dx = self.dx
        else:
            raise TypeError('unsupported operand type(s) for +: %s and %s' % (type(self), type(other)))
        return ADTangent(x, dx)
    
    def __sub__(self, other):
        if isinstance(other, ADTangent):
            x = self.x - other.x
            dx = self.dx - other.dx
        elif isinstance(other, (int, float)):
            x = self.x - other
            dx = self.dx
        else:
            raise TypeError('unsupported operand type(s) for -: %s and %s' % (type(self), type(other)))
        return ADTangent(x, dx)

    def __mul__(self, other):
        if isinstance(other, ADTangent):
            x = self.x * other.x
            dx = self.x * other.dx + self.dx * other.x
        elif isinstance(other, (int, float)):
            x = self.x * other
            dx = self.dx * other
        else:
            raise TypeError('unsupported operand type(s) for *: %s and %s' % (type(self), type(other)))
        return ADTangent(x, dx)
    
    
    def log(self):
        x = np.log(self.x)
        dx = self.dx / self.x
        return ADTangent(x, dx)
    
    def sin(self):
        x = np.sin(self.x)
        dx = self.dx * np.cos(self.x)   
        return ADTangent(x, dx)

In [42]:
x = ADTangent(x = 2., dx = 1)
y = ADTangent(x = 5., dx = 0)

f = ADTangent.log(x) + x * y + ADTangent.sin(y)
print(f)

value: 9.734222905896807, grad: 5.5


In [43]:
# 使用 torch 验证

import torch
from torch.autograd import Variable
x = Variable(torch.tensor(2.0), requires_grad=True)
y = Variable(torch.tensor(5.0), requires_grad=True)
z = torch.log(x) + x * y + torch.sin(y)
z.backward()
print(x.grad, y.grad)

tensor(5.5000) tensor(2.2837)


### 2.后向微分
后向自动微分关键步骤为：
反向模式根据从后向前计算，依次得到对每个中间变量节点的偏导数，直到到达自变量节点处，这样就得到了每个输入的偏导数。在每个节点处，根据该节点的后续节点（前向传播中的后续节点）计算其导数值。

In [44]:
from typing import List, NamedTuple, Callable, Dict, Optional

_name = 1
# 定义一个函数，用于生成新的名字
def fresh_name():
    # 声明一个全局变量，用于存储当前的名字
    global _name
    # 生成新的名字，格式为v加上当前的名字
    name = f'v{_name}'
    # 将当前的名字加1
    _name += 1
    # 返回新的名字
    return name

In [59]:

class Variable:
    # 初始化函数，用于创建一个对象
    def __init__(self, value, name=None):
        # 将传入的value参数赋值给对象的value属性
        self.value = value
        # 如果传入的name参数为空，则调用fresh_name()函数生成一个新的名字，否则将传入的name参数赋值给对象的name属性
        self.name = name or fresh_name()
        
    def __repr__(self):
        # 返回self.value的字符串表示
        return repr(self.value)

    @staticmethod
    # 定义一个函数，用于创建一个常量
    def constant(value, name=None):
        # 创建一个变量对象，传入值和名称
        var = Variable(value, name)
        # 打印变量的名称和值
        print(f'{var.name} = {var.value}')
        # 返回变量对象
        return var
    

    def __mul__(self, other):
        return ops_mul(self, other)
    
    def __add__(self, other):
        return ops_add(self, other)

    def __sub__(self, other):
        return ops_sub(self, other)

    def sin(self):
        return ops_sin(self)
    
    def log(self):
        return ops_log(self)

def ops_mul(self, other):
    # forward
    x = Variable(self.value * other.value)
    print(f'{x.name} = {self.name} * {other.name}')
    
    # backward
    def propagate(dl_doutputs):
        # 计算梯度
        dl_dx, = dl_doutputs
        dx_dother = self # r=self.value * other.value
        dx_self = other # dx_dother = self.value * other.value
        dl_dself = dl_dx * dx_self
        dl_dother = dl_dx * dx_dother
        dl_inputs = [dl_dself, dl_dother]
        return dl_inputs
    # 记录操作的输入输出
    tape = Tape(inputs=[self.name, other.name], outputs=[x.name], propagate=propagate)
    gradient_tape.append(tape)
    return x

def ops_add(self, other):
    # forward
    x = Variable(self.value + other.value)
    print(f'{x.name} = {self.name} + {other.name}')

    # backward
    def propagate(dl_doutputs):
        # 计算梯度
        dl_dx, = dl_doutputs
        dx_dself = Variable(1.0)
        dx_dother = Variable(1.0)
        dl_dself = dl_dx * dx_dself
        dl_dother = dl_dx * dx_dother
        return [dl_dself, dl_dother]
    # 记录操作的输入输出
    tape = Tape(inputs=[self.name, other.name], outputs=[x.name], propagate=propagate)
    gradient_tape.append(tape)
    return x

def ops_sub(self, other):
    # forward
    x = Variable(self.value - other.value)
    print(f'{x.name} = {self.name} - {other.name}')
    
    # backward
    def propagate(dl_doutputs):
        dl_dx, = dl_doutputs
        dx_dself = Variable(1.0)
        dx_dother = Variable(-1.0)
        dl_dself = dl_dx * dx_dself
        dl_dother = dl_dx * dx_dother
        return [dl_dself, dl_dother]
    # 记录操作的输入输出
    tape = Tape(inputs=[self.name, other.name], outputs=[x.name], propagate=propagate)
    gradient_tape.append(tape)
    return x

def ops_sin(self):
    # forward
    x = Variable(np.sin(self.value))
    print(f'{x.name} = sin({self.name})')

    # backward
    def propagate(dl_doutputs):
        dl_dx, = dl_doutputs
        dx_dself = Variable(np.cos(self.value))
        dl_dself = dl_dx * dx_dself
        return [dl_dself]
    
    # 记录操作的输入输出
    tape = Tape(inputs=[self.name], outputs=[x.name], propagate=propagate)
    gradient_tape.append(tape)
    return x

def ops_log(self):
    # forward
    x = Variable(np.log(self.value))
    print(f'{x.name} = log({self.name})')
    
    # backward
    def propagate(dl_doutputs):
        dl_dx, = dl_doutputs
        dx_dself = Variable(1.0 / self.value)
        dl_dself = dl_dx * dx_dself
        return [dl_dself]
    # 记录操作的输入输出
    tape = Tape(inputs=[self.name], outputs=[x.name], propagate=propagate)
    gradient_tape.append(tape)
    return x
    

# 这个类的作用在于记录前向传播过程中产生的中间变量，以及这些中间变量之间的依赖关系，以便于在反向传播过程中计算梯度。
# NamedTuple 是一个元组子类，用于定义不可变的数据结构。在这个类中，我们定义了三个属性：inputs、outputs 和 propagate。
class Tape(NamedTuple):
    # 输入列表
    inputs: List[str]
    # 输出列表
    outputs: List[str]
    # 传播函数，接受一个变量列表，返回一个变量列表
    propagate : 'Callable[List[Variable], List[Variable]]'

# 重置 Tape 的方法 reset_tape，方便运行多次自动微分，每次自动微分过程都会产生 Tape List
gradient_tape : List[Tape] = []

def reset_tape():
    global _name
    _name = 1
    gradient_tape.clear()

In [82]:
def gred(l, results):
    dl_d = {} # dl/dX 的所有梯度
    dl_d[l.name] = Variable(1.)

    print("dl_d", dl_d)
    # 添加 dl/dl = 1
    
    def gather_grads(entrys):
        return [dl_d[entry] if entry in dl_d else None for entry in entrys]
    

    # 反向传播
    for entry in reversed(gradient_tape):
        # 计算梯度
        print(entry)
        dl_outputs = gather_grads(entry.outputs)
        dl_inputs = entry.propagate(dl_outputs)
        
        for input, grad in zip(entry.inputs, dl_inputs):
            if input not in dl_d:
                dl_d[input] = grad
            else:
                dl_d[input] = dl_d[input] + grad
                
    for name, grad in dl_d.items():
        print(f'd{l.name}_d{name} = {grad.value}')
    
    return gather_grads([result.name for result in results])

reset_tape()
x = Variable(2.0, name='v-1')
y = Variable(5.0, name='v0')

f = Variable.log(x) + x * y - Variable.sin(y)
print(f)

v1 = log(v-1)
v2 = v-1 * v0
v3 = v1 + v2
v4 = sin(v0)
v5 = v3 - v4
np.float64(11.652071455223084)


In [83]:
dx, dy = gred(f, [x, y])
print(dx)
print(dy)

dl_d {'v5': 1.0}
Tape(inputs=['v3', 'v4'], outputs=['v5'], propagate=<function ops_sub.<locals>.propagate at 0x000001EC37302EE0>)
v9 = v6 * v7
v10 = v6 * v8
Tape(inputs=['v0'], outputs=['v4'], propagate=<function ops_sin.<locals>.propagate at 0x000001EC37302790>)
v12 = v10 * v11
Tape(inputs=['v1', 'v2'], outputs=['v3'], propagate=<function ops_add.<locals>.propagate at 0x000001EC25759940>)
v15 = v9 * v13
v16 = v9 * v14
Tape(inputs=['v-1', 'v0'], outputs=['v2'], propagate=<function ops_mul.<locals>.propagate at 0x000001EC372D4DC0>)
v17 = v16 * v0
v18 = v16 * v-1
v19 = v12 + v18
Tape(inputs=['v-1'], outputs=['v1'], propagate=<function ops_log.<locals>.propagate at 0x000001EC372D2EE0>)
v21 = v15 * v20
v22 = v17 + v21
dv5_dv5 = 1.0
dv5_dv3 = 1.0
dv5_dv4 = -1.0
dv5_dv0 = 1.7163378145367738
dv5_dv1 = 1.0
dv5_dv2 = 1.0
dv5_dv-1 = 5.5
5.5
np.float64(1.7163378145367738)
